In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # 设置日志级别为ERROR，以减少警告信息
# 禁用 Gemini 的底层库（gRPC 和 Abseil）在初始化日志警告
os.environ["GRPC_VERBOSITY"] = "ERROR"
os.environ["GLOG_minloglevel"] = "3"  # 0: INFO, 1: WARNING, 2: ERROR, 3: FATAL
os.environ["GLOG_minloglevel"] = "true"
import logging
import tensorflow as tf
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
!export TF_FORCE_GPU_ALLOW_GROWTH=true

````{admonition} Copyright 2019 The TensorFlow Authors.
```
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
```
````

# 初学者的 TensorFlow 2.0 教程

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/tutorials/quickstart/beginner"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 观看</a>   </td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/quickstart/beginner.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行 </a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/quickstart/beginner.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 查看源代码</a>   </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tutorials/quickstart/beginner.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a>   </td>
</table>

```{topic} 导航

此简短介绍使用 [Keras](https://tensorflow.google.cn/guide/keras/overview) 进行以下操作：

1. 加载预构建的数据集。
2. 构建对图像进行分类的神经网络机器学习模型。
3. 训练此神经网络。
4. 评估模型的准确率。
```

## 设置 TensorFlow

首先将 TensorFlow 导入到您的程序：

In [2]:
import tensorflow as tf

如果您在自己的开发环境而不是 [Colab](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb) 中操作，请参阅设置 TensorFlow 以进行开发的[安装指南](https://tensorflow.google.cn/install)。

注：如果您使用自己的开发环境，请确保您已升级到最新的 `pip` 以安装 TensorFlow 2 软件包。有关详情，请参阅[安装指南](https://tensorflow.google.cn/install)。

## 加载数据集

加载并准备 [MNIST 数据集](http://yann.lecun.com/exdb/mnist/)。图像的像素值范围从 0 到 255。通过将值除以 `255.0` 来将这些值缩放到 0 到 1 的范围。这还会将样本数据从整数转换为浮点数：

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## 构建机器学习模型

构建 `tf.keras.Sequential` 模型：

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

[`Sequential`](https://tensorflow.google.cn/guide/keras/sequential_model) 对于堆叠层很有用，其中每层都有输入[张量](https://tensorflow.google.cn/guide/tensor)和输出张量。层是具有已知数学结构的函数，可以重复使用并具有可训练的变量。大多数 TensorFlow 模型都由层组成。此模型使用 [`Flatten`](https://tensorflow.google.cn/api_docs/python/tf/keras/layers/Flatten)、[`Dense`](https://tensorflow.google.cn/api_docs/python/tf/keras/layers/Dense) 和 [`Dropout`](https://tensorflow.google.cn/api_docs/python/tf/keras/layers/Dropout) 层。

对于每个样本，模型都会返回包含 [logits](https://developers.google.com/machine-learning/glossary#logits) 或 [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) 分数的向量，每个类一个。

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.29356095, -0.18187018, -0.09060116,  0.02143997,  0.24739322,
         0.02414675, -0.34286305,  0.02412909,  0.39885187,  0.39236996]],
      dtype=float32)

{func}`tensorflow.nn.softmax` 函数将这些 logits 转换为每个类的*概率*： 

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.07084628, 0.07921798, 0.08678835, 0.09707788, 0.12168877,
        0.09734099, 0.06743812, 0.09733927, 0.14158855, 0.14067376]],
      dtype=float32)

注：可以将 {func}`tensorflow.nn.softmax` 烘焙到网络最后一层的激活函数中。虽然这可以使模型输出更易解释，但不建议使用这种方式，因为在使用 `softmax` 输出时不可能为所有模型提供精确且数值稳定的损失计算。 

使用 {class}`~tensorflow.keras.losses.SparseCategoricalCrossentropy` 定义训练的损失函数：

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

损失函数采用真实值向量和逻辑向量，并返回每个样本的标量损失。此损失等于真实类的负对数概率：如果模型确定类正确，则损失为零。

这个未经训练的模型给出的概率接近随机（每个类为 1/10），因此初始损失应该接近 `-tf.math.log(1/10) ~= 2.3`。

In [8]:
loss_fn(y_train[:1], predictions).numpy()

2.329535

在开始训练之前，使用 Keras {meth}`~tensorflow.models.model.Model.compile` 配置和编译模型。将 [`optimizer`](https://tensorflow.google.cn/api_docs/python/tf/keras/optimizers) 类设置为 `adam`，将 `loss` 设置为您之前定义的 `loss_fn` 函数，并通过将 `metrics` 参数设置为 `accuracy` 来指定要为模型评估的指标。

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## 训练并评估模型

使用 `Model.fit` 方法调整您的模型参数并最小化损失： 

In [10]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5


I0000 00:00:1729733723.232793 3171586 service.cc:146] XLA service 0x7f3d74006ef0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729733723.232815 3171586 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
I0000 00:00:1729733723.232833 3171586 service.cc:154]   StreamExecutor device (1): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5


  78/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5239 - loss: 1.5746

I0000 00:00:1729733725.890061 3171586 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.8585 - loss: 0.4892
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9557 - loss: 0.1527
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9675 - loss: 0.1089
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9736 - loss: 0.0847
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9762 - loss: 0.0727


`Model.evaluate` 方法会检查模型的性能（通常是在[验证集](https://developers.google.com/machine-learning/glossary#validation-set)或[测试集](https://developers.google.com/machine-learning/glossary#test-set)上）。

In [11]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 2s - 8ms/step - accuracy: 0.9769 - loss: 0.0729


[0.07293855398893356, 0.9768999814987183]

现在，这个照片分类器的准确度已经达到 98%。想要了解更多，请阅读 [TensorFlow 教程](https://tensorflow.google.cn/tutorials/)。

如果您想让模型返回概率，可以封装经过训练的模型，并将 softmax 附加到该模型：

In [12]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [13]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[5.70330599e-07, 1.30929301e-09, 3.31053479e-06, 6.53220995e-05,
        5.35139099e-13, 1.03895630e-07, 7.61280822e-14, 9.99930143e-01,
        1.48227429e-07, 3.17864817e-07],
       [2.60567834e-08, 1.02092403e-04, 9.99895215e-01, 2.47421417e-06,
        2.45008007e-14, 1.18589675e-07, 7.92316790e-09, 1.31756047e-11,
        1.22220749e-08, 6.07167094e-13],
       [7.65437704e-08, 9.98419642e-01, 1.11594715e-03, 8.62039724e-06,
        1.14041950e-05, 3.91538015e-06, 4.68364533e-07, 3.88111861e-04,
        4.93279695e-05, 2.40418944e-06],
       [9.99776661e-01, 5.56393265e-10, 1.60305080e-05, 3.42057410e-06,
        1.33386193e-05, 5.57819885e-06, 1.63603676e-04, 1.89078055e-05,
        5.46506328e-07, 1.99484361e-06],
       [3.93524680e-08, 6.11409900e-09, 1.89196712e-07, 8.14441758e-09,
        9.99578297e-01, 1.13444706e-07, 4.50963626e-06, 2.70816599e-06,
        2.53426805e-07, 4.13792412e-04]], dtype=float32)>

## 结论

恭喜！您已经利用 [Keras](https://tensorflow.google.cn/guide/keras/overview) API 借助预构建数据集训练了一个机器学习模型。

有关使用 Keras 的更多示例，请查阅[教程](https://tensorflow.google.cn/tutorials/keras/)。要详细了解如何使用 Keras 构建模型，请阅读[指南](https://tensorflow.google.cn/guide/keras)。如果您想详细了解如何加载和准备数据，请参阅有关[图像数据加载](https://tensorflow.google.cn/tutorials/load_data/images)或 [CSV 数据加载](https://tensorflow.google.cn/tutorials/load_data/csv)的教程。
